In [1]:
using DifferentialEquations
using Plots; pyplot()

Plots.PyPlotBackend()

In [2]:
function wetting_ode(du, u, p, t)
    L = u[1]
    l = u[2]
    V₀, m, Δ, γ, ω, η, t₀, Kₚpc = p
    dL = ( 0.1*(4*(V₀-π*m*Δ*l^2)/π)^0.3 * (10*γ*ω/η)^0.1 * 1/(t+t₀)^0.9 
        - 2/3*(π*m*Δ*Kₚpc*L/η)/((V₀-π*m*Δ*l^2)*log(l/L)) )
    dl = (Kₚpc/η)/(l*log(l/L))
    du[1] = dL
    du[2] = dl
end

wetting_ode (generic function with 1 method)

# Solve with guessed parameters

In [7]:
# Parameters
# Initial volume (ICs)
V₀ = 4./3*pi*10.^3 #Å^3
# Porosity (fit)
m = .7 # unitless (estimate)
# Monolayer height (our observation)
Δ = 1.5 # Å
# Surface tension (Vega, table 3)
γ = 6.0e0 # pN/Å
# Lubrication coefficient (fit)
ω = 0.01 # unitless (estimate)
# Viscosity (from Balasubramanian fig. 2)
η = 6.0e-3 # pN-ns/Å^2
# Initial time (ICs)
t₀ = .5 # ns (estimate?)
# Something related to pc (fit)
Kₚpc = 1.e3 # pN (estimate)

p = [V₀, m, Δ, γ, ω, η, t₀, Kₚpc]

# Initial Conditions
L₀ = 15.
l₀ = 15.
u₀ = [L₀, l₀]

# Time
tmax = 1.
tspan = (t₀, tmax);

In [8]:
prob = ODEProblem(wetting_ode, u₀, tspan, p)
sol = solve(prob, Tsit5(), abstol=1e-7, reltol=1e-7)
println("Done")
plot(sol)
println("Plotted")

Done


LoadError: [91mBoundsError: attempt to access 1-element Array{Float64,1} at index [2][39m

# Read data from files

In [ ]:
mr_data = readdlm("../pres_data/mr.txt");
br_data = readdlm("../pres_data/br.txt");
ca_data = readdlm("../pres_data/cos_ca_rad_scaled.txt");
hb_data = readdlm("../pres_data/h.txt");

In [ ]:
sim_num = 5
plot(hb_data[:,2*sim_num-1], hb_data[:,2*sim_num])

# Optimize parameters

In [ ]:
mr_data

In [ ]:
function sol_data_diff(p)
    """Difference between numerical solution and data"""
    # Compare to 40A droplet
    sim_num = 5
    val_col = 2*sim_num
    t_col = val_col - 1
    
    t_data_40A = mr_data[:,t_col]
    mr_data_40A = mr_data[:,val_col]
    t₀ = t_data_40A[1]
    println("t₀ = $t₀")
    prob = ODEProblem(wetting_ode, u₀, tspan, p)
    sol = solve(prob, Tsit5(), abstol=1e-7, reltol=1e-7)
    bh = sol.u[:,1]
    rb = sol.u[:,2]
    t = sol.t
    plot()
    plot!(t_data_40A, mr_data_40A)
end

In [ ]:
sol_data_diff(p)